<br>

## ***Imports and Data Loading***

In [2]:
import numpy as numpy
import pandas as pd
import pickle

import torch
import transformers
from summarizer import Summarizer,TransformerSummarizer
from rouge import Rouge


In [7]:
data_path = "D:\VIT\SEM-7\B2 - NLP\Project\Code\curation-corpus-master\output.csv"

In [8]:
df = pd.read_csv(data_path)
df_1 = df.drop("date",axis=1)
df_1.dropna(inplace=True)
df_1.head()

,title,summary,url,article_content
0,Tencent gains approval to sell mutual funds to...,Tencent has been granted a licence from the Ch...,http://www.scmp.com/business/companies/article...,Traditional finance houses now being seriously...
1,"India testing blockchains in education, health...",India is testing blockchain applications in ed...,https://www.vccircle.com/niti-aayog-explores-b...,"The Indian government's policy think tank, Nit..."
2,Higher living wage risks robot takeover of low...,The UK Institute for Fiscal Studies has warned...,https://news.sky.com/story/ifs-living-wage-inc...,Increases in minimum wage levels risk raising ...
3,Regus WeWork may seek stock-market flotation t...,Co-working start-up WeWork may go public this ...,https://www.fool.com/investing/2018/01/03/will...,"Depending on whom you ask, WeWork is either a ..."
4,AMD poised to gain market share as Intel pound...,Intel has seen more than $11bn wiped off its m...,https://www.cnbc.com/2018/01/05/amd-is-big-win...,AMD is big winner from chip flaw fiasco as mor...


In [9]:
print("Number of records: ", len(df_1))
print("Null records:\n", df_1.isnull().sum(axis = 0))

Number of records:  37639
Null records:
 title              0
summary            0
url                0
article_content    0
dtype: int64


In [10]:
art_content = df_1["article_content"].tolist()

### 37,639 articles with no null values

<br>

---

<br>

## ***BART Summarization***

In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


In [6]:
torch_device = 'cpu'
#torch_device = torch.device("cuda")

In [7]:
def bart_summarize(text):
  
  text = text.replace('\n','')
  text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
  summary_ids = model.generate(text_input_ids)           
  summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary_txt

In [ ]:
bart_summary = []
for article in art_content[:100]:
    bart_summary.append(bart_summarize(article))

In [34]:
 # Save summaries
 with open('./saves/bart_summary.pkl', 'wb') as f:
    pickle.dump(bart_summary, f)

In [7]:
# Load summaries into list
with open('saves/bart_summary.pkl', 'rb') as f:
    bart_summary = pickle.load(f)
print(len(bart_summary))

100


In [31]:
rouge = Rouge() 
bart_results = rouge.get_scores(bart_summary, art_content[:100], avg=True)
print(bart_results['rouge-1']['f'])

0.3425688880978472


In [32]:
print(bart_results)

{'rouge-1': {'r': 0.24376196727960603, 'p': 0.95256595316306, 'f': 0.3425688880978472}, 'rouge-2': {'r': 0.18992568657754003, 'p': 0.8877813022847721, 'f': 0.2626904085860145}, 'rouge-l': {'r': 0.24333340306696757, 'p': 0.9497327716956646, 'f': 0.34184003199002516}}


In [33]:
#Save ROUGE Results
with open('./saves/bart_results.pkl', 'wb') as f:
    pickle.dump(bart_results, f)

In [17]:
# Load BART ROUGE results 
with open('saves/bart_results.pkl', 'rb') as f:
    bart_results = pickle.load(f)
print(len(bart_results))

3


---

<br>

## ***T5 Model testing***

In [14]:
from transformers import pipeline
import os

In [15]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

ValueError: Pipeline cannot infer suitable model classes from t5-base

In [ ]:
summary_text = summarizer(text, do_sample=False)[0]['summary_text']
print(summary_text)

---

<br>

## ***BERT Model***

In [15]:
bert_summary = []
bert_model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [16]:
for article in art_content[:100]:
    bert_summary.append(''.join(bert_model(article, min_length=60)))

D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\e

In [28]:
#Save BERT Summaries
with open('./saves/bert_summary.pkl', 'wb') as f:
    pickle.dump(bert_summary, f)

In [ ]:
# Load BERT summaries into list
with open('./saves/bert_summary.pkl', 'rb') as f:
    bert_summary = pickle.load(f)
print(len(bert_summary))

In [18]:
rouge = Rouge()
bert_results = rouge.get_scores(bert_summary, art_content[:100], avg=True)
print(bert_results['rouge-1']['f'])

0.512665142711876


In [19]:
print(bert_results)

{'rouge-1': {'r': 0.35671353293714303, 'p': 0.9987867380091021, 'f': 0.512665142711876}, 'rouge-2': {'r': 0.2884779593282122, 'p': 0.9808698819649082, 'f': 0.4293938466441541}, 'rouge-l': {'r': 0.35671353293714303, 'p': 0.9987867380091021, 'f': 0.512665142711876}}


In [27]:
with open('./saves/bert_result.pkl', 'wb') as f:
    pickle.dump(bert_results, f)

In [20]:
# Load BERT ROUGE results 
with open('saves/bert_result.pkl', 'rb') as f:
    bert_results = pickle.load(f)
print(len(bert_results))

3


---

<br>

## ***GPT-2*** 

In [14]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

Downloading: 100%|██████████| 718/718 [00:00<00:00, 144kB/s]
Downloading: 100%|██████████| 1.42G/1.42G [02:45<00:00, 9.20MB/s]
Downloading: 100%|██████████| 0.99M/0.99M [00:01<00:00, 734kB/s]
Downloading: 100%|██████████| 446k/446k [00:01<00:00, 422kB/s]
Downloading: 100%|██████████| 1.29M/1.29M [00:01<00:00, 813kB/s] 


In [20]:
gpt2_summary = []
for article in art_content[:100]:
    gpt2_summary.append(''.join(GPT2_model(article, min_length=60)))

D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\e

In [26]:
# Save GPT2 Summaries
with open('./saves/gpt2_summary.pkl', 'wb') as f:
    pickle.dump(gpt2_summary, f)

In [ ]:
# Load GPT2 summaries into list
with open('./saves/gpt2_summary.pkl', 'rb') as f:
    gpt2_summary = pickle.load(f)
print(len(gpt2_summary))

In [22]:
rouge = Rouge()
gpt2_results = rouge.get_scores(gpt2_summary, art_content[:100], avg=True)
print(gpt2_results['rouge-1']['f'])

0.5003818950060116


In [23]:
print(gpt2_results)

{'rouge-1': {'r': 0.346430328700958, 'p': 0.9986815098013179, 'f': 0.5003818950060116}, 'rouge-2': {'r': 0.2784888887918763, 'p': 0.9790739542634641, 'f': 0.4160236755825082}, 'rouge-l': {'r': 0.346430328700958, 'p': 0.9986815098013179, 'f': 0.5003818950060116}}


In [25]:
with open('./saves/gpt2_result.pkl', 'wb') as f:
    pickle.dump(gpt2_results, f)

In [21]:
# Load BERT ROUGE results 
with open('saves/gpt2_result.pkl', 'rb') as f:
    gpt2_results = pickle.load(f)
print(len(gpt2_results))

3


---

<br>

## ***XLNet Summarization***

In [7]:
xlnet_model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 779k/779k [00:01<00:00, 401kB/s]
Downloading: 100%|██████████| 1.32M/1.32M [00:02<00:00, 486kB/s]


In [10]:
xlnet_summary = []
for article in art_content[:100]:
    xlnet_summary.append(''.join(xlnet_model(article, min_length=60)))

D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\envs\abstractive\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "
D:\Anaconda\e

In [11]:
# Save XLNet Summaries
with open('./saves/xlnet_summary.pkl', 'wb') as f:
    pickle.dump(xlnet_summary, f)

In [12]:
# Load XLNet summaries into list
with open('./saves/xlnet_summary.pkl', 'rb') as f:
    xlnet_summary = pickle.load(f)
print(len(xlnet_summary))

100


In [13]:
rouge = Rouge()
xlnet_results = rouge.get_scores(xlnet_summary, art_content[:100], avg=True)
print(xlnet_results['rouge-1']['f'])

0.5126260066401299


In [14]:
print(xlnet_results)

{'rouge-1': {'r': 0.36282156276483435, 'p': 0.9984175768160023, 'f': 0.5126260066401299}, 'rouge-2': {'r': 0.2964653852175559, 'p': 0.9773250014001291, 'f': 0.42994633608829863}, 'rouge-l': {'r': 0.36282156276483435, 'p': 0.9984175768160023, 'f': 0.5126260066401299}}


In [15]:
with open('./saves/xlnet_result.pkl', 'wb') as f:
    pickle.dump(xlnet_results, f)

In [22]:
# Load XLNet ROUGE results 
with open('saves/xlnet_result.pkl', 'rb') as f:
    xlnet_results = pickle.load(f)
print(len(xlnet_results))

3


---

<br>

## ***PEGASUS Summarization***

In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
from rouge import Rouge

In [2]:
model_name = "google/pegasus-large"
torch_device = "cuda" if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Downloading: 100%|██████████| 1.82M/1.82M [00:01<00:00, 1.06MB/s]
Downloading: 100%|██████████| 65.0/65.0 [00:00<00:00, 8.34kB/s]
Downloading: 100%|██████████| 88.0/88.0 [00:00<00:00, 11.0kB/s]
Downloading: 100%|██████████| 3.02k/3.02k [00:00<?, ?B/s]
Downloading: 100%|██████████| 2.12G/2.12G [06:24<00:00, 5.91MB/s]


In [11]:
pegasus_summary = []
for article in art_content[:100]:
    batch = tokenizer(article, truncation=True, padding='longest', return_tensors="pt").to(torch_device)
    translated = model.generate(**batch)
    pegasus_summary.append(tokenizer.batch_decode(translated, skip_special_token=True)[0])
print(len(pegasus_summary))

100


In [12]:
# Save PEGASUS Summaries
with open('./saves/pegasus_summary.pkl', 'wb') as f:
    pickle.dump(pegasus_summary, f)

In [ ]:
 # Load PEGASUS summaries into list
with open('./saves/pegasus_summary.pkl', 'rb') as f:
    pegasus_summary = pickle.load(f)
print(len(pegasus_summary))

In [13]:
rouge = Rouge()
pegasus_results = rouge.get_scores(pegasus_summary, art_content[:100], avg=True)
print(pegasus_results['rouge-1']['f'])

0.39022612843617793


In [14]:
print(pegasus_results)

{'rouge-1': {'r': 0.2573194659980407, 'p': 0.9968122245958115, 'f': 0.39022612843617793}, 'rouge-2': {'r': 0.20230428093218447, 'p': 0.985872266417941, 'f': 0.3152113315304677}, 'rouge-l': {'r': 0.2573194659980407, 'p': 0.9968122245958115, 'f': 0.39022612843617793}}


In [15]:
with open('./saves/pegasus_results.pkl', 'wb') as f:
    pickle.dump(pegasus_results, f)

In [ ]:
# Load PEGASUS ROUGE results 
with open('saves/pegasus_results.pkl', 'rb') as f:
    pegasus_results = pickle.load(f)
print(len(pegasus_results))

---

<br>

## ***Results*** 



### ***Loading results***

In [3]:
# Load BART ROUGE results 
with open('saves/bart_results.pkl', 'rb') as f:
    bart_results = pickle.load(f)
print(len(bart_results))

3


In [4]:
# Load BERT ROUGE results 
with open('saves/bert_result.pkl', 'rb') as f:
    bert_results = pickle.load(f)
print(len(bert_results))

3


In [5]:
# Load GPT2 ROUGE results 
with open('saves/gpt2_result.pkl', 'rb') as f:
    gpt2_results = pickle.load(f)
print(len(gpt2_results))

3


In [6]:
# Load XLNet ROUGE results 
with open('saves/xlnet_result.pkl', 'rb') as f:
    xlnet_results = pickle.load(f)
print(len(xlnet_results))

3


In [7]:
# Load PEGASUS ROUGE results 
with open('saves/pegasus_results.pkl', 'rb') as f:
    pegasus_results = pickle.load(f)
print(len(pegasus_results))

3


### ***Compiling Scores***

In [8]:
rouge_1_scores = [bart_results['rouge-1'],bert_results['rouge-1'],gpt2_results['rouge-1'],xlnet_results['rouge-1'], pegasus_results['rouge-1']]
rouge_2_scores = [bart_results['rouge-2'],bert_results['rouge-2'],gpt2_results['rouge-2'],xlnet_results['rouge-2'], pegasus_results['rouge-2']]
rouge_l_scores = [bart_results['rouge-l'],bert_results['rouge-l'],gpt2_results['rouge-l'],xlnet_results['rouge-l'], pegasus_results['rouge-l']]

In [9]:
model_names = ["BERT", "BART", "GPT2", "XLNet","PEGASUS"]
rouge_1_recall = [rouge_1_scores[0]['r'], rouge_1_scores[1]['r'], rouge_1_scores[2]['r'], rouge_1_scores[3]['r'], rouge_1_scores[4]['r']]
rouge_1_precision = [rouge_1_scores[0]['p'], rouge_1_scores[1]['p'], rouge_1_scores[2]['p'],rouge_1_scores[3]['p'], rouge_1_scores[4]['p']]
rouge_1_f = [rouge_1_scores[0]['f'], rouge_1_scores[1]['f'], rouge_1_scores[2]['f'], rouge_1_scores[3]['f'], rouge_1_scores[4]['f']]

for i in range(len(rouge_1_recall)):
    rouge_1_recall[i] = round(rouge_1_recall[i],4)
    rouge_1_precision[i] = round(rouge_1_precision[i],4)
    rouge_1_f[i] = round(rouge_1_f[i],4)


data_1 = {
    'Recall': rouge_1_recall,
    'Precision': rouge_1_precision,
    'F-Score': rouge_1_f 
}

In [10]:
rouge_2_recall = [rouge_2_scores[0]['r'], rouge_2_scores[1]['r'], rouge_2_scores[2]['r'], rouge_2_scores[3]['r'], rouge_2_scores[4]['r']]
rouge_2_precision = [rouge_2_scores[0]['p'], rouge_2_scores[1]['p'], rouge_2_scores[2]['p'],rouge_2_scores[3]['p'], rouge_2_scores[4]['p']]
rouge_2_f = [rouge_2_scores[0]['f'], rouge_2_scores[1]['f'], rouge_2_scores[2]['f'], rouge_2_scores[3]['f'], rouge_2_scores[4]['f']]


for i in range(len(rouge_1_recall)):
    rouge_2_recall[i] = round(rouge_1_recall[i],4)
    rouge_2_precision[i] = round(rouge_1_precision[i],4)
    rouge_2_f[i] = round(rouge_1_f[i],4)

data_2 = {
    'Recall': rouge_2_recall,
    'Precision': rouge_2_precision,
    'F-Score': rouge_2_f 
}

In [11]:
rouge_l_recall = [rouge_l_scores[0]['r'], rouge_l_scores[1]['r'], rouge_l_scores[2]['r'], rouge_l_scores[3]['r'], rouge_l_scores[4]['r']]
rouge_l_precision = [rouge_l_scores[0]['p'], rouge_l_scores[1]['p'], rouge_l_scores[2]['p'],rouge_l_scores[3]['p'], rouge_l_scores[4]['p']]
rouge_l_f = [rouge_l_scores[0]['f'], rouge_l_scores[1]['f'], rouge_l_scores[2]['f'], rouge_l_scores[3]['f'], rouge_l_scores[4]['f']]

"""
for i in range(len(rouge_l_recall)):
    rouge_l_recall[i] = round(rouge_l_recall[i],4)
    rouge_l_precision[i] = round(rouge_l_precision[i],4)
    rouge_l_f[i] = round(rouge_l_f[i],4)
"""
data_l = {
    'Recall': rouge_l_recall,
    'Precision': rouge_l_precision,
    'F-Score': rouge_l_f 
}

In [12]:
final_results_1 = pd.DataFrame(data_1, index=model_names)
final_results_1.style.highlight_max(color = 'green', axis = 0)
#final_results_1.head()

,Recall,Precision,F-Score
BERT,0.243800,0.952600,0.342600
BART,0.356700,0.998800,0.512700
GPT2,0.346400,0.998700,0.500400
XLNet,0.362800,0.998400,0.512600
PEGASUS,0.257300,0.996800,0.390200


In [13]:
final_results_2 = pd.DataFrame(data_2, index=model_names)
final_results_2.style.highlight_max(color = 'green', axis = 0)

,Recall,Precision,F-Score
BERT,0.243800,0.952600,0.342600
BART,0.356700,0.998800,0.512700
GPT2,0.346400,0.998700,0.500400
XLNet,0.362800,0.998400,0.512600
PEGASUS,0.257300,0.996800,0.390200


In [47]:
final_results_l = pd.DataFrame(data_l, index=model_names)
final_results_l.style.highlight_max(color = 'green', axis = 0)

,Recall,Precision,F-Score
BERT,0.243333,0.949733,0.341840
BART,0.356714,0.998787,0.512665
GPT2,0.346430,0.998682,0.500382
XLNet,0.362822,0.998418,0.512626
PEGASUS,0.257319,0.996812,0.390226
